# Conversational Task Agent

Taskbot that assist users in multi-step tasks, such as baking a 
birthday  cake  or  fixing  a  scratch  on  a  car  —  and  adapt  those  instructions  based  on  the 
resources  and  tools  available  to  the  customer.  If,  for  example,  a  customer  ran  out  of  an 
ingredient  halfway  through  a  recipe  or  didn’t  have  a  specific  tool  for  a  DIY  project,  the 
taskbot had to adjust the plan and suggest possible solutions.

## Phase 1 - (Task Retriever): Implementing a search index for recipes with OpenSearch

In [2]:
from modules.Indexing import Indexing

indexer = Indexing()

indexer.calculateAndStoreEmbeddings()
print("Embeddings stored and encoded")
indexer.createOpenSearchMappings()
print("Mappings created")
indexer.readAndStoreRecipesFromFile("./receitas.json")
print("Recipes stored")

Embeddings stored and encoded
Index already existed. Nothing to be done.
Mappings created
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
create

In [5]:
from modules.Search import Search

search = Search()

# search.SearchTitleTxt("chicken")
# search.SearchDescriptionTxt("Chicken")
# search.SearchTitleEmbeddings("Chicken")
# search.SearchDescriptionEmbeddings("Chicken")
# search.SearchRecipeTime(30, 5)
# search.SearchRecipeIngredients("chicken, rice")
# search.SearchRecipeDifficulty("easy")
# search.SearchRecipeNutrition("protein")

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [10]:
from opensearchpy import OpenSearch

host = 'api.novasearch.org'
port = 443
user = 'user212'
password = 'soO-2518'
index_name = user
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = 600
    )
    print('\nDeleting index:')
    print(response)


Deleting index:
{'acknowledged': True}
